This agent takes advantage of board wrapping and goes for the closest food. It also avoids moving in the same direction as the previous direction which ends the game for the goose. 

Additionally, it actively avoids other geese and itself as well as avoiding situations whereby it will be trapped by it's own body. Also, if faced with a situation where there are no valid moves (i.e will hit itself or the head of another goose if going for same food) then it will take the risk rather than deafult to the deafult direction which is South.

It will also try to move in circle patterns to survive longer which seems to be the meta currently.

In [ ]:
import random
import numpy as np
from itertools import chain
from collections import defaultdict
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
previous_direction  = ""
banned_directions = {"EAST" : "WEST", "WEST" : "EAST", "NORTH" : "SOUTH", "SOUTH" : "NORTH", "" : ""}

circle_directions = {"SOUTH" : "WEST", "NORTH" : "EAST", "EAST" : "SOUTH", "WEST" : "NORTH", "" : ""}

backup_directions = {"SOUTH" : "EAST", "EAST" : "NORTH", "WEST" : "SOUTH", "NORTH" : "WEST", "" : ""}

#Cant move in previous direction

def BFS(graphdict,source):
    
    #initialisation
    colour={}
    distance={}
    predecessor={}
    for v in graphdict.keys():
        colour[v]='w'
        distance[v]=np.inf
        predecessor[v]=None
        
    colour[source]='g'
    distance[source]=0
    queue=[source]
    
    #iteration
    while queue != []:
        u=queue[0]
        for vertex in graphdict[u]:
            if colour[vertex]=='w':
                colour[vertex]='g'
                distance[vertex]=distance[u]+1
                predecessor[vertex]=u
                queue.append(vertex)
        colour[u]='b'
        
        if len(queue)>1:
            queue=queue[1:]
        else:
            queue=[]
            
    return (distance,predecessor)

def pathfind(graphdict,start,finish):
    (d,p)=BFS(graphdict,start)
   # print(p)
    thepath=[finish]
    if finish != start:
        running=1
        v=finish
        #print("V",v)
        while running==1:
           # print("PV", p[v])
            v=p[v]
            thepath=[v]+thepath
            if v==start:
                running=0
    return thepath

def addEdge(u,v, graph):
        graph[u].append(v)
        return graph

def getgraph():
    graph = defaultdict(list)
    for i in range(77):
        directions = {"NORTH" : [-1, 0], "SOUTH" : [1, 0], "EAST" : [0,1], "WEST" : [0,-1]}
        for direction in ["NORTH", "EAST", "SOUTH", "WEST"]:
            row, col = row_col(i, 11)
            add_row, add_col = directions[direction]
            new_row = (row + add_row) % 7
            new_col = (col + add_col) % 11
            newpos = new_row*11 + new_col 
            oldpos = row*11 + col
            graph = addEdge(oldpos,newpos,graph)
    return graph


def removeEdge(u,v, graph):
    if v in graph[u]:
        graph[u].remove(v)
    return graph

def removepositions(my_goose, graph):
    for i in range(len(my_goose)-1, -1, -1):
        graph = removeEdge(my_goose[i], my_goose[i-1], graph)
    return graph

def get_direction(from_, to_, configuration):
    
    fromrow, fromcol = row_col(from_, configuration.columns)
    torow, tocol = row_col(to_, configuration.columns)
    
    
    row_difference = abs(torow - fromrow)
    col_difference = abs(tocol - fromcol)

    other_row_difference = 7 - row_difference
    other_col_difference = 11 - col_difference

    if torow > fromrow:
        if row_difference > other_row_difference:
            direction = "NORTH"
        elif row_difference < other_row_difference:
            direction = "SOUTH"
    if torow < fromrow:
        if row_difference < other_row_difference:
            direction = "NORTH"
        elif row_difference > other_row_difference:
            direction = "SOUTH"

    if tocol > fromcol:
        if col_difference < other_col_difference:
            direction = "EAST"
        elif col_difference > other_col_difference:
            direction = "WEST"
    if tocol < fromcol:
        if col_difference > other_col_difference:
            direction = "EAST"
        elif col_difference < other_col_difference:
            direction = "WEST"
    
    return direction

def get_board():
    board = np.zeros((7,11), int)
    count = 0
    for i in range(len(board)):
        for j in range(len(board[1])):
            board[i][j] = count
            count +=1
    return board

def get_preferred_moves(player_goose, observation, configuration):
    possible_moves_ = possible_moves(player_goose, observation, configuration)
    player_row, player_column = row_col(player_goose, configuration.columns)
    
    food_row, food_col = get_closest_food(player_row, player_column, observation, configuration)
    food_pos = food_row*11 + food_col
    
    distances = [get_distance(player_pos, food_pos, configuration, observation) for player_pos in possible_moves_]
    preferred_moves = [possible_moves_[idx] for idx in np.argsort(distances)] 
    
    return preferred_moves


def get_new_dangerous(player_goose, move, player_pos, dangerous_geese_positions, previous_direction, observation, configuration):
    new_goose = new_goose_position(player_goose, move)
    new_goose_head = new_goose[0]
    count = 0
    
    temp_previous_direction = get_direction(player_pos, move, configuration)
  
    
    new_possible_moves = possible_moves(new_goose_head, observation, configuration)
    for val in new_possible_moves:
        intended_direction = get_direction(new_goose_head, val, configuration)
        if intended_direction == banned_directions[previous_direction]:
            count += 1
        if val in dangerous_geese_positions or val in new_goose:
            count += 1
        
    return new_goose_head, count, temp_previous_direction, new_goose


def dead_end_detection(player_goose, player_pos, move, dangerous_geese_positions, previous_direction, observation, configuration, depth):
  #  print("Depth", depth)
    if depth == 1:
        new_goose_head, count, temp_previous_direction, new_goose = get_new_dangerous(player_goose, move, player_pos, dangerous_geese_positions, previous_direction, observation, configuration)
        return count
    
    else:
        for i in range(depth):
            new_goose_head, count, temp_previous_direction, new_goose = get_new_dangerous(player_goose, move, player_pos, dangerous_geese_positions, previous_direction, observation, configuration)
            if count == 4:
                print("------")
                print("Count hit 4, printing description")
                print("Move", move)
                print("Depth", depth)
                print(new_goose, new_goose_head)
                print("------")
                return count
            new_preferred_moves = get_preferred_moves(new_goose_head, observation, configuration)
            for newmove in new_preferred_moves:
                new_goose_head, count, temp_previous_direction, new_goose = get_new_dangerous(new_goose, newmove, new_goose_head, dangerous_geese_positions, temp_previous_direction, observation, configuration)
            player_pos = new_goose_head
            player_goose = new_goose
    return count
    
def get_closest_food(player_row, player_column, observation, configuration):
    foods = observation.food
    distances = []
    for i in range(len(foods)):
        food_row, food_column = row_col(foods[i], configuration.columns)
        total_distance = (food_row - player_row)**2 + (food_column - player_column)**2
        distances.append(total_distance)
    min_distance_index = distances.index(min(distances))
    closest_food = foods[min_distance_index]
    food_row, food_column = row_col(foods[min_distance_index], configuration.columns)
    return food_row, food_column

def possible_moves(position, observation, configuration):
    possible_moves  = [] 
    directions = {"NORTH" : [-1, 0], "SOUTH" : [1, 0], "EAST" : [0,1], "WEST" : [0,-1]}
    row, col = row_col(position, configuration.columns)
    for direction in list(directions.keys()):
        add_row, add_col = directions[direction]
        new_row = (row + add_row) % 7
        new_col = (col + add_col) % 11
        rowcol_topos = new_row*11 + new_col
        possible_moves.append(rowcol_topos)
    return possible_moves


def get_next_placement(goose, observation, configuration):
    if not goose:
        return set()
    
    # Retrieve basic information for goose
    goose_head = goose[0]
    rows, columns = configuration.rows, configuration.columns
    
    # Get all possible moves
    possible_moves_ = possible_moves(goose_head, observation, configuration)
    # Placement of goose is equal to current placement and one cell towards food
    next_move_placement = set(goose + possible_moves_)

    return next_move_placement

def get_distance(position, destination, configuration, observation):
    distances = []
    row, col = row_col(position, configuration.columns)
    destrow, destcol = row_col(destination, configuration.columns)
    total_distance = (destrow - row)**2 + (destcol - col)**2
    return total_distance

def new_goose_position(goose, move):
    copy_goose = goose
   # print(copy_goose)
    copy_goose = copy_goose[-1:] + copy_goose[:-1]
    copy_goose[0] = move
    return copy_goose

def new_position(position, direction, observation, configuration):
    directions = {"NORTH" : [-1, 0], "SOUTH" : [1, 0], "EAST" : [0,1], "WEST" : [0,-1]}
    row, col = row_col(position, configuration.columns)
    add_row, add_col = directions[direction]
    new_row = (row + add_row) % 7
    new_col = (col + add_col) % 11
    rowcol_topos = new_row*11 + new_col
    return rowcol_topos

def myround(x, base=40):
    return base * round(x/base)



    
def agent(obs_dict, config_dict):
   
    global previous_direction
    
    
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
#     print("Index", player_index)
    
#     print("Observation.step", observation.step+1)
    player_goose = observation.geese[player_index]
    player_pos = player_goose[0]
    player_row, player_column = row_col(player_pos, configuration.columns)
    
    geese_positions = observation.geese[:]
    geese_positions.pop(player_index)
    dangerous_geese_positions = [get_next_placement(goose, observation, configuration)
                             for goose in geese_positions]
    dangerous_geese_positions = list(chain.from_iterable(dangerous_geese_positions))
    new_dangerous = [[i] for i in dangerous_geese_positions]
    
    two_step = [get_next_placement(goose, observation, configuration)
                             for goose in new_dangerous]
    two_step = list(chain.from_iterable(two_step))
    own_next_placement = get_next_placement(player_goose, observation, configuration)
    

    closest_starve_step = myround(observation.step)
    steps_until_starve = closest_starve_step - observation.step
    

    graph = getgraph()
    graph = removepositions(player_goose, graph)
    if previous_direction != "" and len(player_goose) > 1 and observation.step <= 150:
        disregard = False
        intended_direction = circle_directions[previous_direction]
        move = new_position(player_goose[0], intended_direction, observation, configuration)
        graph = removeEdge(move, player_pos, graph)
        path_list = pathfind(dict(graph),move,player_goose[-1])[:-1]
        
        for pos in path_list:
            if pos in dangerous_geese_positions and pos in player_goose or intended_direction == banned_directions[previous_direction]:
                disregard = True
        
        if disregard:
            intended_direction = backup_directions[intended_direction]
            move = new_position(player_goose[0], intended_direction, observation, configuration)
            graph = getgraph()
            graph = removepositions(player_goose, graph)  
            graph = removeEdge(move, player_pos, graph)
            path_list = pathfind(dict(graph),move,player_goose[-1])[:-1]
            for pos in path_list:
                if pos in dangerous_geese_positions and pos in player_goose or intended_direction == banned_directions[previous_direction]:
                    disregard_backup = True
                
                if diregard_backup == False:
                    previous_direction = intended_direction
                    return intened_direction
        else:
            previous_direction = intended_direction
            return intended_direction
        
      
        
        
#         if new_rowcol not in dangerous_geese_positions and new_rowcol not in player_goose and intended_direction != banned_directions[previous_direction]:
#                 previous_direction = intended_direction
#                 return intended_direction
#         else:
#             intended_direction = backup_directions[intended_direction]
#             new_rowcol = new_position(player_goose[0], intended_direction, observation, configuration)
#             if new_rowcol not in dangerous_geese_positions and new_rowcol not in player_goose and intended_direction != banned_directions[previous_direction]:
#                 previous_direction = intended_direction
#                 return intended_direction
    
    food_row, food_col = get_closest_food(player_row, player_column, observation, configuration)
    food_pos = food_row*11 + food_col
    
    possible_moves_ = possible_moves(player_pos, observation, configuration)
    
    
    
    
#     print("Step", observation.step +1)
#     print("player index", player_index)
#     print("player goose", player_goose)
#     print("Possible moves", possible_moves_)

# #         banned_direction = banned_directions[previous_direction]
# #         last_position = new_position(player_pos, banned_direction, observation, configuration)
# #         graph = removeEdge(player_pos, last_position, graph)
# #      #   print(graph)
#    # print("Graph",dict(graph))
    
    
    
    distances = [get_distance(player_pos, food_pos, configuration, observation) for player_pos in possible_moves_]
    preferred_moves = [possible_moves_[idx] for idx in np.argsort(distances)]
    
    
    graph = getgraph()
    graph = removepositions(player_goose, graph)
    
    temp_direction = None
    chosen_direction = "SOUTH"
    
    for move in preferred_moves:
        
        graph = removeEdge(move, player_pos, graph)
        path_list = pathfind(dict(graph),move,player_goose[-1])[:-1]
        graph = getgraph()
        graph = removepositions(player_goose, graph)
        for pos in path_list:
            if pos in dangerous_geese_positions or pos in player_goose:
                continue
    
        
        
#         count = dead_end_detection(player_goose, player_pos, move, dangerous_geese_positions, previous_direction, observation, configuration, depth = 5)
# #         print("Move", move)
#         print("Count",count)
   
    
#         if count == 4:
#             continue
        
        if move not in dangerous_geese_positions and move not in player_goose:
            intended_direction = get_direction(player_pos, move,configuration)
            
            if intended_direction != banned_directions[previous_direction]:
                chosen_direction = intended_direction
                temp_direction = intended_direction
                break
    
    if temp_direction is None:
        for move in preferred_moves:
            if move in player_goose:
                continue
            else:
                temp_direction = get_direction(player_pos, move, configuration)

                if temp_direction != banned_directions[previous_direction]:
                    chosen_direction = temp_direction
                    break
    
    
 
    previous_direction = chosen_direction 
    return chosen_direction

In [ ]:
from kaggle_environments import evaluate, make, utils

# Setup a hungry_geese environment.
env = make("hungry_geese", debug = True)
env.run([agent, agent])


In [ ]:
env.render(mode="ipython", width=700, height=700)

# Greedy VS Greedy

In [ ]:
def mean_reward(rewards):
    print(rewards)
    return sum(r[0] for r in rewards) / float(len(rewards))

# Run multiple episodes to estimate its performance.
print("Greedy vs Random Agent:", mean_reward(evaluate("hungry_geese", [agent, "random"], num_episodes=100)))
print("Greedy vs Greedy:", mean_reward(evaluate("hungry_geese", [agent, agent], num_episodes=100)))

## Write Submission File

In [ ]:
%%writefile agent.py

import random
import numpy as np
from itertools import chain
from collections import defaultdict
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
previous_direction  = ""
banned_directions = {"EAST" : "WEST", "WEST" : "EAST", "NORTH" : "SOUTH", "SOUTH" : "NORTH", "" : ""}

circle_directions = {"SOUTH" : "WEST", "NORTH" : "EAST", "EAST" : "SOUTH", "WEST" : "NORTH", "" : ""}

backup_directions = {"SOUTH" : "EAST", "EAST" : "NORTH", "WEST" : "SOUTH", "NORTH" : "WEST", "" : ""}

#Cant move in previous direction

def BFS(graphdict,source):
    
    #initialisation
    colour={}
    distance={}
    predecessor={}
    for v in graphdict.keys():
        colour[v]='w'
        distance[v]=np.inf
        predecessor[v]=None
        
    colour[source]='g'
    distance[source]=0
    queue=[source]
    
    #iteration
    while queue != []:
        u=queue[0]
        for vertex in graphdict[u]:
            if colour[vertex]=='w':
                colour[vertex]='g'
                distance[vertex]=distance[u]+1
                predecessor[vertex]=u
                queue.append(vertex)
        colour[u]='b'
        
        if len(queue)>1:
            queue=queue[1:]
        else:
            queue=[]
            
    return (distance,predecessor)

def pathfind(graphdict,start,finish):
    (d,p)=BFS(graphdict,start)
   # print(p)
    thepath=[finish]
    if finish != start:
        running=1
        v=finish
        #print("V",v)
        while running==1:
           # print("PV", p[v])
            v=p[v]
            thepath=[v]+thepath
            if v==start:
                running=0
    return thepath

def addEdge(u,v, graph):
        graph[u].append(v)
        return graph

def getgraph():
    graph = defaultdict(list)
    for i in range(77):
        directions = {"NORTH" : [-1, 0], "SOUTH" : [1, 0], "EAST" : [0,1], "WEST" : [0,-1]}
        for direction in ["NORTH", "EAST", "SOUTH", "WEST"]:
            row, col = row_col(i, 11)
            add_row, add_col = directions[direction]
            new_row = (row + add_row) % 7
            new_col = (col + add_col) % 11
            newpos = new_row*11 + new_col 
            oldpos = row*11 + col
            graph = addEdge(oldpos,newpos,graph)
    return graph


def removeEdge(u,v, graph):
    if v in graph[u]:
        graph[u].remove(v)
    return graph

def removepositions(my_goose, graph):
    for i in range(len(my_goose)-1, -1, -1):
        graph = removeEdge(my_goose[i], my_goose[i-1], graph)
    return graph

def get_direction(from_, to_, configuration):
    
    fromrow, fromcol = row_col(from_, configuration.columns)
    torow, tocol = row_col(to_, configuration.columns)
    
    
    row_difference = abs(torow - fromrow)
    col_difference = abs(tocol - fromcol)

    other_row_difference = 7 - row_difference
    other_col_difference = 11 - col_difference

    if torow > fromrow:
        if row_difference > other_row_difference:
            direction = "NORTH"
        elif row_difference < other_row_difference:
            direction = "SOUTH"
    if torow < fromrow:
        if row_difference < other_row_difference:
            direction = "NORTH"
        elif row_difference > other_row_difference:
            direction = "SOUTH"

    if tocol > fromcol:
        if col_difference < other_col_difference:
            direction = "EAST"
        elif col_difference > other_col_difference:
            direction = "WEST"
    if tocol < fromcol:
        if col_difference > other_col_difference:
            direction = "EAST"
        elif col_difference < other_col_difference:
            direction = "WEST"
    
    return direction

def get_board():
    board = np.zeros((7,11), int)
    count = 0
    for i in range(len(board)):
        for j in range(len(board[1])):
            board[i][j] = count
            count +=1
    return board

def get_preferred_moves(player_goose, observation, configuration):
    possible_moves_ = possible_moves(player_goose, observation, configuration)
    player_row, player_column = row_col(player_goose, configuration.columns)
    
    food_row, food_col = get_closest_food(player_row, player_column, observation, configuration)
    food_pos = food_row*11 + food_col
    
    distances = [get_distance(player_pos, food_pos, configuration, observation) for player_pos in possible_moves_]
    preferred_moves = [possible_moves_[idx] for idx in np.argsort(distances)] 
    
    return preferred_moves


def get_new_dangerous(player_goose, move, player_pos, dangerous_geese_positions, previous_direction, observation, configuration):
    new_goose = new_goose_position(player_goose, move)
    new_goose_head = new_goose[0]
    count = 0
    
    temp_previous_direction = get_direction(player_pos, move, configuration)
  
    
    new_possible_moves = possible_moves(new_goose_head, observation, configuration)
    for val in new_possible_moves:
        intended_direction = get_direction(new_goose_head, val, configuration)
        if intended_direction == banned_directions[previous_direction]:
            count += 1
        if val in dangerous_geese_positions or val in new_goose:
            count += 1
        
    return new_goose_head, count, temp_previous_direction, new_goose


def dead_end_detection(player_goose, player_pos, move, dangerous_geese_positions, previous_direction, observation, configuration, depth):
  #  print("Depth", depth)
    if depth == 1:
        new_goose_head, count, temp_previous_direction, new_goose = get_new_dangerous(player_goose, move, player_pos, dangerous_geese_positions, previous_direction, observation, configuration)
        return count
    
    else:
        for i in range(depth):
            new_goose_head, count, temp_previous_direction, new_goose = get_new_dangerous(player_goose, move, player_pos, dangerous_geese_positions, previous_direction, observation, configuration)
            if count == 4:
                print("------")
                print("Count hit 4, printing description")
                print("Move", move)
                print("Depth", depth)
                print(new_goose, new_goose_head)
                print("------")
                return count
            new_preferred_moves = get_preferred_moves(new_goose_head, observation, configuration)
            for newmove in new_preferred_moves:
                new_goose_head, count, temp_previous_direction, new_goose = get_new_dangerous(new_goose, newmove, new_goose_head, dangerous_geese_positions, temp_previous_direction, observation, configuration)
            player_pos = new_goose_head
            player_goose = new_goose
    return count
    
def get_closest_food(player_row, player_column, observation, configuration):
    foods = observation.food
    distances = []
    for i in range(len(foods)):
        food_row, food_column = row_col(foods[i], configuration.columns)
        total_distance = (food_row - player_row)**2 + (food_column - player_column)**2
        distances.append(total_distance)
    min_distance_index = distances.index(min(distances))
    closest_food = foods[min_distance_index]
    food_row, food_column = row_col(foods[min_distance_index], configuration.columns)
    return food_row, food_column

def possible_moves(position, observation, configuration):
    possible_moves  = [] 
    directions = {"NORTH" : [-1, 0], "SOUTH" : [1, 0], "EAST" : [0,1], "WEST" : [0,-1]}
    row, col = row_col(position, configuration.columns)
    for direction in list(directions.keys()):
        add_row, add_col = directions[direction]
        new_row = (row + add_row) % 7
        new_col = (col + add_col) % 11
        rowcol_topos = new_row*11 + new_col
        possible_moves.append(rowcol_topos)
    return possible_moves


def get_next_placement(goose, observation, configuration):
    if not goose:
        return set()
    
    # Retrieve basic information for goose
    goose_head = goose[0]
    rows, columns = configuration.rows, configuration.columns
    
    # Get all possible moves
    possible_moves_ = possible_moves(goose_head, observation, configuration)
    # Placement of goose is equal to current placement and one cell towards food
    next_move_placement = set(goose + possible_moves_)

    return next_move_placement

def get_distance(position, destination, configuration, observation):
    distances = []
    row, col = row_col(position, configuration.columns)
    destrow, destcol = row_col(destination, configuration.columns)
    total_distance = (destrow - row)**2 + (destcol - col)**2
    return total_distance

def new_goose_position(goose, move):
    copy_goose = goose
   # print(copy_goose)
    copy_goose = copy_goose[-1:] + copy_goose[:-1]
    copy_goose[0] = move
    return copy_goose

def new_position(position, direction, observation, configuration):
    directions = {"NORTH" : [-1, 0], "SOUTH" : [1, 0], "EAST" : [0,1], "WEST" : [0,-1]}
    row, col = row_col(position, configuration.columns)
    add_row, add_col = directions[direction]
    new_row = (row + add_row) % 7
    new_col = (col + add_col) % 11
    rowcol_topos = new_row*11 + new_col
    return rowcol_topos

def myround(x, base=40):
    return base * round(x/base)



    
def agent(obs_dict, config_dict):
   
    global previous_direction
    
    
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
#     print("Index", player_index)
    
#     print("Observation.step", observation.step+1)
    player_goose = observation.geese[player_index]
    player_pos = player_goose[0]
    player_row, player_column = row_col(player_pos, configuration.columns)
    
    geese_positions = observation.geese[:]
    geese_positions.pop(player_index)
    dangerous_geese_positions = [get_next_placement(goose, observation, configuration)
                             for goose in geese_positions]
    dangerous_geese_positions = list(chain.from_iterable(dangerous_geese_positions))
    new_dangerous = [[i] for i in dangerous_geese_positions]
    
    two_step = [get_next_placement(goose, observation, configuration)
                             for goose in new_dangerous]
    two_step = list(chain.from_iterable(two_step))
    own_next_placement = get_next_placement(player_goose, observation, configuration)
    

    closest_starve_step = myround(observation.step)
    steps_until_starve = closest_starve_step - observation.step
    

    graph = getgraph()
    graph = removepositions(player_goose, graph)
    if previous_direction != "" and len(player_goose) > 1 and observation.step <= 150:
        disregard = False
        intended_direction = circle_directions[previous_direction]
        move = new_position(player_goose[0], intended_direction, observation, configuration)
        graph = removeEdge(move, player_pos, graph)
        path_list = pathfind(dict(graph),move,player_goose[-1])[:-1]
        
        for pos in path_list:
            if pos in dangerous_geese_positions and pos in player_goose or intended_direction == banned_directions[previous_direction]:
                disregard = True
        
        if disregard:
            intended_direction = backup_directions[intended_direction]
            move = new_position(player_goose[0], intended_direction, observation, configuration)
            graph = getgraph()
            graph = removepositions(player_goose, graph)  
            graph = removeEdge(move, player_pos, graph)
            path_list = pathfind(dict(graph),move,player_goose[-1])[:-1]
            for pos in path_list:
                if pos in dangerous_geese_positions and pos in player_goose or intended_direction == banned_directions[previous_direction]:
                    disregard_backup = True
                
                if diregard_backup == False:
                    previous_direction = intended_direction
                    return intened_direction
        else:
            previous_direction = intended_direction
            return intended_direction
        
      
        
        
#         if new_rowcol not in dangerous_geese_positions and new_rowcol not in player_goose and intended_direction != banned_directions[previous_direction]:
#                 previous_direction = intended_direction
#                 return intended_direction
#         else:
#             intended_direction = backup_directions[intended_direction]
#             new_rowcol = new_position(player_goose[0], intended_direction, observation, configuration)
#             if new_rowcol not in dangerous_geese_positions and new_rowcol not in player_goose and intended_direction != banned_directions[previous_direction]:
#                 previous_direction = intended_direction
#                 return intended_direction
    
    food_row, food_col = get_closest_food(player_row, player_column, observation, configuration)
    food_pos = food_row*11 + food_col
    
    possible_moves_ = possible_moves(player_pos, observation, configuration)
    
    
    
    
#     print("Step", observation.step +1)
#     print("player index", player_index)
#     print("player goose", player_goose)
#     print("Possible moves", possible_moves_)

# #         banned_direction = banned_directions[previous_direction]
# #         last_position = new_position(player_pos, banned_direction, observation, configuration)
# #         graph = removeEdge(player_pos, last_position, graph)
# #      #   print(graph)
#    # print("Graph",dict(graph))
    
    
    
    distances = [get_distance(player_pos, food_pos, configuration, observation) for player_pos in possible_moves_]
    preferred_moves = [possible_moves_[idx] for idx in np.argsort(distances)]
    
    
    graph = getgraph()
    graph = removepositions(player_goose, graph)
    
    temp_direction = None
    chosen_direction = "SOUTH"
    
    for move in preferred_moves:
        
        graph = removeEdge(move, player_pos, graph)
        path_list = pathfind(dict(graph),move,player_goose[-1])[:-1]
        graph = getgraph()
        graph = removepositions(player_goose, graph)
        for pos in path_list:
            if pos in dangerous_geese_positions or pos in player_goose:
                continue
    
        
        
#         count = dead_end_detection(player_goose, player_pos, move, dangerous_geese_positions, previous_direction, observation, configuration, depth = 5)
# #         print("Move", move)
#         print("Count",count)
   
    
#         if count == 4:
#             continue
        
        if move not in dangerous_geese_positions and move not in player_goose:
            intended_direction = get_direction(player_pos, move,configuration)
            
            if intended_direction != banned_directions[previous_direction]:
                chosen_direction = intended_direction
                temp_direction = intended_direction
                break
    
    if temp_direction is None:
        for move in preferred_moves:
            if move in player_goose:
                continue
            else:
                temp_direction = get_direction(player_pos, move, configuration)

                if temp_direction != banned_directions[previous_direction]:
                    chosen_direction = temp_direction
                    break
    
    
 
    previous_direction = chosen_direction 
    return chosen_direction